# Teste Apache Spark - Willian Henrique Barbosa Rocha
<hr>

## Qual o objetivo do comando cache em Spark?
O comando cache tem como objetivo colocar os data-sets in memory atraves dos cluster, para que o acesso a esses dados fiquem mais rapidos sem a necessidade de ler novamente em disco. Caso nao seja possivel alocar o data-set na memoria,o Spark ira fazer o split desses dados no disco.

## O mesmo código implementado em Spark é normalmente mais rápido que a implementação equivalente em MapReduce. Por quê?
O Spark foi concebido para trabalhar com os dados in-memory, ou seja, utilizando o acesso a memoria, enquanto o Hadoop utiliza-se do acesso ao disco para processar as consultas aos dados. Porem o hardware para processar Spark precisa ser mais robusto, devido ao maior utilizacao de memoria.

## Qual é a função do *SparkContext* ?
O SparkContext representa a conexao, um objeto do cluster Spark, e pode ser utilizado para criar os RDDs, acumuladores e variáveis de broadcast no cluster Spark.

## Explique com suas palavras o  que é Resilient Distributed Datasets (RDD).
RDD's são as estruturas de dados fundamentais do Spark, distribuições imutáveis e tolerantes a falhas das coleções de objetos que são paralelizados através do cluster. Existem 2 tipos de operações: Transformations, Actions.

## *GroupByKey* é menos eficiente que *reduceByKey* em grandes dataset. Por quê?
O comando **GroupByKey** realiza o Suffle dos dados através da rede para formar a lista (K, V), isso é particularmente custoso quando estamos trabalhando com um conjunto de dados de grande volume, entretanto o comando **reduceByKey** realiza o suffle dos dados somente nos resultados da operação reduce, isso resultado em significante redução de tráfico na rede.

## Explique o  que o  código Scala abaixo faz.
```scala
val textFile = sc.textFile("hdfs://...")
val counts   = textFile.flatMap(line => line.split(""))
               .map(word => (word,1))
               .reduceByKey(_ + _)
counts.saveAsTextFile("hdfs://...")
```

Codigo de word count, realiza o mapping das palavras, e a realiza a contagem de quantas vezes aquela palavra apareceu.

## HTTP requests to the NASA Kennedy Space Center WWW server
<hr>

**Fonte oficial do dateset:** [http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html)

**Dados:**
- [Jul​ ​ 01​ ​ to​ ​ Jul​ ​ 31,​ ​ ASCII​ ​ format,​ ​ 20.7​ ​ MB​ ​ gzip​ ​ compressed](ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz)​ , ​ ​ 205.2​ ​ MB.
- [Aug​ ​ 04​ ​ to​ ​ Aug​ ​ 31,​ ​ ASCII​ ​ format,​ ​ 21.8​ ​ MB​ ​ gzip​ ​ compressed](ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz)​ , ​ ​ 167.8​ ​ MB.

**Sobre o dataset**​ : Esses dois conjuntos de dados possuem todas as requisições HTTP para o servidor da NASA Kennedy
Space​ ​ Center​ ​ WWW​ ​ na​ ​ Flórida​ ​ para​ ​ um​ ​ período​ ​ específico.

Os​ ​ logs​ ​ estão​ ​ em​ ​ arquivos​ ​ ASCII​ ​ com​ ​ uma​ ​ linha​ ​ por​ ​ requisição​ ​ com​ ​ as​ ​ seguintes​ ​ colunas:
- **Host fazendo a requisição**​ . Um hostname quando possível, caso contrário o endereço de internet se o nome
não​ ​ puder​ ​ ser​ ​ identificado.
- **Timestamp**​ ​ no​ ​ formato​ ​ "DIA/MÊS/ANO:HH:MM:SS​ ​ TIMEZONE"
- **Requisição​ ​ (entre​ ​ aspas)**
- **Código​ ​ do​ ​ retorno​ ​ HTTP**
- **Total​ ​ de​ ​ bytes​ ​ retornados**

### Questões
Responda​ ​ as​ ​ seguintes​ ​ questões​ ​ devem​ ​ ser​ ​ desenvolvidas​ ​ em​ ​ Spark​ ​ utilizando​ ​ a ​ ​ sua​ ​ linguagem​ ​ de​ ​ preferência.

[Manual de referência do SparkSQL](https://spark.apache.org/docs/latest/sql-programming-guide.html)

In [8]:
# Configurando a sessao spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Teste Técnico de Apache Spark").getOrCreate()

In [110]:
# https://spark.apache.org/docs/2.2.1/sql-programming-guide.html#generic-loadsave-functions
df = spark.sql("SELECT * FROM csv.`teste`")

In [111]:
df.head()

Row(_c0='www-c1.proxy.aol.com - - [01/Aug/1995:00:28:34 -0400] "GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0" 200 13450')

In [124]:
import pyspark.sql.functions as f
from pyspark.sql.functions import trim
from pyspark.sql.functions import regexp_replace

In [113]:
df = df.select(
        "_c0",
        f.split("_c0", " - - ")[0].alias("host_requisicao"),
        f.split("_c0", " - - ")[1].alias("_c1")
    )

In [114]:
df.head()

Row(_c0='www-c1.proxy.aol.com - - [01/Aug/1995:00:28:34 -0400] "GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0" 200 13450', host_requisicao='www-c1.proxy.aol.com', _c1='[01/Aug/1995:00:28:34 -0400] "GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0" 200 13450')

In [115]:
df = df.select("host_requisicao",
         f.split(f.regexp_replace("_c1","\[([^]]+)\]",r"$1,"),",")[0].alias("timestamp_requisicao"),
         f.split(f.regexp_replace("_c1","\[([^]]+)\]",r"$1,"),",")[1].alias("_c2"))

In [116]:
df.head()

Row(host_requisicao='www-c1.proxy.aol.com', timestamp_requisicao='01/Aug/1995:00:28:34 -0400', _c2=' "GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0" 200 13450')

In [117]:
df = df.select("host_requisicao","timestamp_requisicao",
         trim(f.split(f.regexp_replace("_c2","\"([^]]+)\"",r"$1,"),",")[0]).alias("requisicao"),
         trim(f.split(f.regexp_replace("_c2","\"([^]]+)\"",r"$1,"),",")[1]).alias("_c3"))

In [118]:
df.head()

Row(host_requisicao='www-c1.proxy.aol.com', timestamp_requisicao='01/Aug/1995:00:28:34 -0400', requisicao='GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0', _c3='200 13450')

In [127]:
df = df.select("host_requisicao","timestamp_requisicao","requisicao",
         (f.split("_c3", " ")[0]).alias("codigo_retorno_http"),
         regexp_replace(f.split("_c3", " ")[1],"-","0").alias("bytes_retorno"))

In [128]:
df.head(5)

[Row(host_requisicao='www-c1.proxy.aol.com', timestamp_requisicao='01/Aug/1995:00:28:34 -0400', requisicao='GET /shuttle/missions/sts-71/mission-sts-71.html HTTP/1.0', codigo_retorno_http='200', bytes_retorno='13450'),
 Row(host_requisicao='line10.pm1.abb.mindlink.net', timestamp_requisicao='01/Aug/1995:00:28:39 -0400', requisicao='GET /images/NASA-logosmall.gif HTTP/1.0', codigo_retorno_http='200', bytes_retorno='786'),
 Row(host_requisicao='line10.pm1.abb.mindlink.net', timestamp_requisicao='01/Aug/1995:00:28:41 -0400', requisicao='GET /images/KSC-logosmall.gif HTTP/1.0', codigo_retorno_http='200', bytes_retorno='1204'),
 Row(host_requisicao='tia1.eskimo.com', timestamp_requisicao='01/Aug/1995:00:28:41 -0400', requisicao='GET /pub/winvn/release.txt HTTP/1.0', codigo_retorno_http='404', bytes_retorno='0')]

#### 1. Número​ ​ de​ ​ hosts​ ​ únicos.

In [7]:
df_jul = spark.read.load("access_log_Jul95.txt")

Py4JJavaError: An error occurred while calling o112.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:290)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:538)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:611)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:603)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:801)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(R

In [5]:
df.createGlobalTempView("people")
sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

